In [33]:
import os
import nltk
import base64

from unstructured.partition.pdf import partition_pdf

from langchain_core.retrievers import BaseRetriever
from langchain_core.documents import Document
from langchain_core.callbacks import CallbackManagerForRetrieverRun
from langchain_core.stores import BaseStore
from pydantic import Field
from typing import List

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage
from langchain_core.stores import InMemoryStore
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_chroma import Chroma

In [2]:
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

In [3]:
file_path = './data/mal_sample.pdf'

In [ ]:
raw_pdf_elements = partition_pdf(
    filename=file_path,
    extract_images_in_pdf=True,
    infer_table_structure=True,
    chunking_strategy='by_title',
    extract_image_block_output_dir='./data/mal_sample_picture'
)

In [5]:
tables = []
texts = []

for element in raw_pdf_elements:
    if 'unstructured.documents.elements.Table' in str(type(element)):
        tables.append(str(element))
    elif 'unstructured.documents.elements.CompositeElement' in str(type(element)):
        texts.append(str(element))

In [6]:
print(tables[0])
print('-----------')
print(tables[1])
print('-----------')
print(texts[0])
print('-----------')
print(texts[1])

202314 (1. 1.~12. 31.) A CHHI(4278) 20234 (1. 1.~10. 21.) (1. 202414 1.~10. 19.) Ail 673 (100) 663 (100.0) 630 (100.0) 5.0% ae tt 569 (84.5) 560 (84.5) 526 (83.5) 16.1% OfAt 104 (15.5) 108 (15.5) 104 (16.5) 1.0% we 0-94] 5 (0.7) 5 (0.8) 2( 0.3) 460.0% 10-194 31 ( 4.6) 30 ( 4.5) 20 ( 3.2) A33.3% 20-29M 201 (29.9) 200 (30.2) 209 (33.2) 4.5% 30-39M| 111 (16.5) 110 (16.6) 90 (14.3) A18.2% 40-494 107 (15.9) 104 (15.7) 96 (15.2) AT.7%
-----------
50-59M| 60-69M| (17.5) 63 ( 9.4) (17.3) 62 ( 9.4) (15.7) 73 (11.6) 17.7% JOH) Olt 37 ( 4.2) 37 ( 5.6) 41 ( 6.5) 10.8% 70¥)0|2f coc 50-594 40-494] 320-394 20-298 10-194] 0-94 200 EOL 180 160 140 == Ee —EEa i Yl | 120 100 80 60 40 20 0 was(m) = | 20 TOMS ae esi 0-594 50-594 40-4944 30-394 10-194] 0-34 200 180 160 140 120 100 #0 60 aa(8) [Ea a ——T —==y 20-255 40 20 nn mE l 0 20 (202314 4225) (202414 4228)
-----------
42주차 (10.13.~10.19.)

| Suyztay | aygises zt | Bail

주차

국내발생 해외유입

전체
-----------
1주~42주 665명 630명 35명 말라리아 환자 발생 현황 ’24년 1주부터 42주까지 말라리

In [9]:
prompt_text = '''당신은 표와 텍스트를 요약하여 검색할 수 있도록 돕는 역할을 맡은 어시스턴트입니다.
이 요약은 임베딩되어 원본 텍스트나 표 요소를 검색하는 데 사용될 것입니다.
표 또는 텍스트에 대한 간결한 요약을 제공하여 검색에 최적화된 형태로 만들어 주세요.
표 또는 텍스트: {element}'''

prompt = ChatPromptTemplate.from_template(prompt_text)
model = ChatOpenAI(model='gpt-5-nano', temperature=0)
summarize_chain = (
    {'element': lambda x: x}
    | prompt
    | model
    | StrOutputParser()
)

In [10]:
text_summaries = summarize_chain.batch(texts, {'max_concurrency':5})
table_summaries = summarize_chain.batch(tables, {'max_concurrency':5})

In [11]:
print(table_summaries[0])
print('---------')
print(table_summaries[1])
print('---------')
print(text_summaries[0])
print('---------')
print(text_summaries[1])

다음은 검색 임베딩용으로 간결하게 요약한 형태입니다.

- 데이터 구성
  - 기간: 3개 시점의 표로 구성됨(표기상 예: 202314, 20234, 202414).
  - 항목: Ail, ae tt, OfAt, we, 10-194, 연령대 구간(20-29M, 30-39M, 40-49), 등 여러 열에 걸친 카운트와 비율 표기.

- 핵심 수치(주요 열과 기간별 값)
  - Ail: 673(100%), 663(100.0%), 630(100.0) / 변화 약 5.0%
  - ae tt: 569(84.5%), 560(84.5%), 526(83.5%) / 변화 약 16.1%
  - OfAt: 104(15.5%), 108(15.5%), 104(16.5%) / 변화 약 1.0%
  - we 0-94: 5(0.7%), 5(0.8%), 2(0.3%) / 변화 약 460.0%
  - 10-194: 31(4.6%), 30(4.5%), 20(3.2%) / 변화 약 33.3%
  - 20-29M: 201(29.9%), 200(30.2%), 209(33.2%) / 증가 +4.5%
  - 30-39M: 111(16.5%), 110(16.6%), 90(14.3%) / 감소 -18.2%
  - 40-49: 107(15.9%), 104(15.7%), 96(15.2%) / 변화 약 -7.0%

- 관찰 포인트
  - 20-29M 비중이 증가 추세(29.9% → 30.2% → 33.2%).
  - 30-39M 비중은 감소 추세(16.5% → 16.6% → 14.3%).
  - 40-49 구간도 소폭 감소(15.9% → 15.7% → 15.2%).

- 주의/메모
  - 원문에 OCR 오타나 표기 불일치가 있어 일부 항목은 해석에 차이가 있을 수 있습니다(예: 40-494 → 40-49, 일부 알파벳 약어의 의미 불확실성).
  - 기간별로 '합계/비율/증감' 형식의 수치가 함께 제시되어 있으므로 임베딩 시 핵심 키워드로는 기간, 연령대, 남성(M 여부), 카운트 및 백분율, 변화율 등을 활용

In [13]:
def encode_image(image_path):
    with open(image_path, 'rb') as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

In [14]:
img_base64_list = []
img_dir_path = './data/mal_sample_picture'

for img_file in sorted(os.listdir(img_dir_path)):
    if img_file.endswith('.jpg') or img_file.endswith('png'):
        img_path = os.path.join(img_dir_path, img_file)
        base64_image = encode_image(img_path)
        img_base64_list.append(base64_image)

In [17]:
def image_summarize(img_base64: str) -> str:
    chat = ChatOpenAI(model='gpt-5-nano', max_completion_tokens=1024)
    prompt = '''당신은 이미지를 요약하여 검색을 위해 사용할 수 있도록 돕는 어시스턴트입니다.
    이 요약은 임베딩되어 원본 이미지를 검색하는 데 사용됩니다.
    이미지 검색에 최적화된 간결한 요약을 작성하세요.'''

    msg = chat.invoke(
        [HumanMessage(
            content=[{'type':'text', 'text':prompt},
                     {'type':'image_url', 'image_url':{'url':f'data:image/jpeg;base64,{img_base64}'}}]
        )]
    )

    return msg.content

In [18]:
image_summaries = []

for img_base64 in img_base64_list:
    image_summary = image_summarize(img_base64)
    image_summaries.append(image_summary)

In [24]:
for image_summary in image_summaries:
    print(image_summary)
    print('-------------------')


-------------------
요약: 중앙 흰색 재생 삼각형 아이콘이 있는 파란-보라색 그라데이션 원형 버튼.

키워드:
- 재생 아이콘
- 원형 버튼
- 그라데이션(파란-보라)
- UI 아이콘
- 미디어 컨트롤
- 플레이 버튼
- 삼각형 디자인
- 비디오/오디오 재생 UI
-------------------

-------------------

-------------------

-------------------

-------------------

-------------------
원형 블루-보라 그라데이션 배경의 흰색 삼각형 플레이 아이콘(비디오 재생 버튼).
-------------------
원형 파란색 그라데이션의 재생 버튼 아이콘으로, 중앙에 흰색 삼각형이 있는 영상 재생 UI 요소. 
키워드: 재생 버튼, 원형 아이콘, 파란 그라데이션, 흰색 삼각형, 영상 재생, 비디오 아이콘
-------------------

-------------------
원형 파란-보라 그라데이션의 흰색 재생 삼각형 아이콘, 비디오 재생 버튼(UI 요소).
-------------------
요약(검색용 alt 텍스트):
- 파란색 그라데이션 원형 버튼 안에 흰색 재생(삼각형) 아이콘이 있는 UI 아이콘.

키워드:
- play button, blue circle, gradient, white triangle, UI icon, media control, vector graphic, round button, flat design, light background
-------------------

-------------------

-------------------
요약: 파란색 원형 재생 아이콘(흰색 삼각형)으로 표시된 미디어 재생 버튼.

키워드: 파란 원형 아이콘, 재생 버튼, 흰 삼각형, 비디오 아이콘, 미디어 컨트롤, UI 아이콘, 플레이 버튼, 원형 버튼, 파란색 그래디언트
-------------------

---------------

In [31]:
class MultiVectorRetriever(BaseRetriever):
    vectorstore: any = Field(description="요약본을 검색할 벡터 저장소")
    docstore: BaseStore = Field(description="원본 문서를 저장하는 키-값 저장소")
    id_key: str = Field(default="doc_id", description="문서 ID를 저장한 메타데이터 키")

    def _get_relevant_documents(
        self, query: str, *, run_manager: CallbackManagerForRetrieverRun
    ) -> List[Document]:
        # 1. 벡터 스토어에서 요약본을 검색합니다. (유사도 검색)
        sub_docs = self.vectorstore.similarity_search(query)
        
        # 2. 검색된 요약본들의 메타데이터에서 'doc_id'를 추출합니다.
        ids = []
        for d in sub_docs:
            if self.id_key in d.metadata:
                ids.append(d.metadata[self.id_key])
        
        # 3. 추출한 ID를 사용해 docstore에서 '원본 문서'를 가져옵니다.
        # (mget은 여러 개의 ID로 한 번에 값을 가져오는 메서드입니다)
        docs = self.docstore.mget(ids)
        
        # 4. None(찾지 못한 경우)을 제외하고 반환합니다.
        return [d for d in docs if d is not None]

In [34]:
vectordb = Chroma(collection_name='multi_modal_rag', embedding_function=OpenAIEmbeddings())
docstore = InMemoryStore()
id_key = 'doc_id'

In [35]:
retriever = MultiVectorRetriever(
    vectorstore=vectordb,
    docstore=docstore,
    id_key=id_key
)